In [ ]:
import requests
import time
import pandas as pd
import numpy as np
import re
import json
import pdb
from bs4 import BeautifulSoup
import personal_keys

# fav_funcs.py

## fav_request
makes request to API, returns json file of user

In [ ]:
def fav_request(username, page_size, page):
    favs_url = 'https://api.ravelry.com/people/{}/favorites/list.json'.format(username)
    favs = requests.get(favs_url, auth = (personal_keys.username(),personal_keys.password()),
                        params={'page_size':page_size, 'page':page})
    return favs.json()

### example output:

In [9]:
fav_request("katec125", 2, 0).keys()

dict_keys(['favorites', 'paginator'])

## create_fav_list
input fav json, output list of fav pattern codes

In [10]:
def create_fav_list(fav_request):
    fav_list = []
    favorites = fav_request['favorites']
    for item in range(0,len(favorites)):
        if favorites[item]['favorited'] is not None:
            if 'pattern_id' in favorites[item]['favorited'].keys():
                fav_list.append(favorites[item]['favorited']['pattern_id'])
            elif 'id' in favorites[item]['favorited'].keys():
                fav_list.append(favorites[item]['favorited']['id'])
    return [code for code in fav_list if code is not None]

### example output: 

In [11]:
fav_json = fav_request("katec125", 2, 0)
create_fav_list(fav_json)

[1017571, 777372]

## get_favs_list
* Input is username, output is list of pattern ids in a person's favorites
* Essentially combination of fav request and create_fav_list
* If the user has over 500 favorites, then only returns the first 500. This is done so other functions don't take forever

In [12]:
def get_favs_list(username):
    favs = fav_request(username, 100, 1)
    fav_list = create_fav_list(favs)
    if favs['paginator']['page_count']>1:
        page_number = 2
        if favs['paginator']['last_page'] > 5:
            last_page = 5
        else:
            last_page = favs['paginator']['last_page']
        while page_number <= last_page:
            new_request_favs = fav_request(username,100, page_number)
            fav_list.extend(create_fav_list(new_request_favs))
            page_number += 1
    return fav_list

### Example output 

In [14]:
get_favs_list("katec125")[:10]

[1017571,
 777372,
 1002216,
 998422,
 877616,
 990541,
 966823,
 444597,
 888409,
 420814]

## fav_dict
* Returns a dict that is just pattern code and 1 for every value
* This is done so we can then do the proj rating func to create user data column

In [15]:
def fav_dict (username):
    fav_list = get_favs_list(username)
    return {code:1 for code in fav_list}

In [21]:
fav_dict("katec125")[1017571]

1

# proj_funcs.py

## proj_json
returns json file with all the info about user patterns

In [22]:
def proj_json(username):
    projects_url = 'https://api.ravelry.com/projects/{}/list.json'.format(username)
    projects = requests.get(projects_url, 
                        auth = (personal_keys.username(),personal_keys.password()))
    return projects.json()

### Example output:

In [23]:
proj_json("katec125").keys()

dict_keys(['projects', 'collections', 'paginator'])

## project_list
Input is a json obtained in the proj_json function, output is a list of pattern ids for user's projects

In [24]:
def project_list(p_json):
    proj_list = []
    proj_list.extend([p_json['projects'][item]['pattern_id'] 
                    for item in range(0,len(p_json['projects']))])
    return proj_list

## get_project_list_from_username

## project_rating

## user_data 

# friend_funcs.py

# search_functions.py 

# yarn_weights.py

# pattern_attr_funcs.py

# count_df_funcs.py
